# A Simple CSV Loader

In [1]:
%matplotlib inline

import dg.conn
con = dg.conn.Conn("host=localhost dbname=ftian")
print (con.ver)

4


In [2]:
from dg.dsutil.csv import CsvXt

In [3]:
# Iris data set is probably the single most used dataset.
# From the sepal/petal lengh/width, can you guess the iris class?
csv = CsvXt('http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data') 
csv.add_col('sepal_length', 'float')
csv.add_col('sepal_width', 'float') 
csv.add_col('petal_length', 'float') 
csv.add_col('petal_width', 'float') 
csv.add_col('iris_class', 'text')
xt = csv.xtable(con)

In [4]:
# I don't know how, but I can check some basics.  At least let's see if the
# avg is significant.   Amateur's first take.   Hmm, Let me guess -- 
# petal size will be enough to tell setosa apart, but virginica
# can versicolor will be harder 
import dg.xtable
avgt = dg.xtable.fromQuery(con, """
    select iris_class, avg(sepal_length) as asl, avg(sepal_width) as asd,
                       avg(petal_length) as apl, avg(petal_width) as apd
    from #0# 
    group by iris_class
""", inputs=[xt])
print(avgt.show())

+-----------------+-------+-------+-------+-------+
| iris_class      |   asl |   asd |   apl |   apd |
|-----------------+-------+-------+-------+-------|
| Iris-virginica  | 6.588 | 2.974 | 5.552 | 2.026 |
| Iris-setosa     | 5.006 | 3.418 | 1.464 | 0.244 |
| Iris-versicolor | 5.936 | 2.77  | 4.26  | 1.326 |
+-----------------+-------+-------+-------+-------+


In [5]:
# Ever execution of the xtable cause internet traffic.
# CTAS.
csv.ctas(con, "iris", distributed_by='iris_class')

In [6]:
localtab = dg.xtable.fromTable(con, 'iris')
print(localtab.show(tablefmt='fancy_grid'))

╒════════════════╤═══════════════╤════════════════╤═══════════════╤═════════════════╕
│   sepal_length │   sepal_width │   petal_length │   petal_width │ iris_class      │
╞════════════════╪═══════════════╪════════════════╪═══════════════╪═════════════════╡
│            7   │           3.2 │            4.7 │           1.4 │ Iris-versicolor │
├────────────────┼───────────────┼────────────────┼───────────────┼─────────────────┤
│            6.4 │           3.2 │            4.5 │           1.5 │ Iris-versicolor │
├────────────────┼───────────────┼────────────────┼───────────────┼─────────────────┤
│            6.9 │           3.1 │            4.9 │           1.5 │ Iris-versicolor │
├────────────────┼───────────────┼────────────────┼───────────────┼─────────────────┤
│            5.5 │           2.3 │            4   │           1.3 │ Iris-versicolor │
├────────────────┼───────────────┼────────────────┼───────────────┼─────────────────┤
│            6.5 │           2.8 │            4.6 │   